# Video Recognition exercise

## Configuring the environment

In [1]:
import cv2
from ultralytics import YOLO
import math 
import os # file 

# MODEL loading and config
# model
model = YOLO("../models/best_from_darin.pt")

class_names = ['Hardhat','Mask','NO-Hardhat',
              'NO-Mask','NO-Safety Vest','Person',
              'Safety Cone','Safety Vest','Machinery','Vehicle']

# Create output folder
os.makedirs("../datasets/stream_result", exist_ok=True)


In [2]:
# DRAWING METHODS

# Function to get class colors
def getColours(cls_num):
    base_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    color_index = cls_num % len(base_colors)
    increments = [(1, -2, 1), (-2, 1, -1), (1, -1, 2)]
    color = [base_colors[color_index][i] + increments[color_index][i] * 
    (cls_num // len(base_colors)) % 256 for i in range(3)]
    return tuple(color)


def drawPredictions(predictions, img, class_names): 
    for p in predictions:
        boxes = p.boxes
        for box in boxes:
            if box.conf[0] > 0.4:
                # get coordinates
                [x1, y1, x2, y2] = box.xyxy[0]
                # convert to int
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
                cls = int(box.cls[0]) # get the class index
        
                colour = getColours(cls)
        
                # draw prediction the rectangle
                cv2.rectangle(img, (x1, y1), (x2, y2), colour, 2)
        
                # put the class name and confidence on the image
                cv2.putText(img, f'{class_names[cls]} {box.conf[0]:.2f}', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, colour, 2)


def predict_video_stream(camera, output, verbose=False, showWindow=True, flipImage=False):
    while True:
        ret, frame = camera.read()
        if not ret:
            break
        
        if flipImage:
            frame = cv2.flip(frame, 1)  
            
        results = model(frame, stream=True, verbose=False)
        drawPredictions(results, frame, class_names)
        
        # Write the frame to the output file
        output.write(frame)
        # Display the captured frame - Performance changes when not showing
        if showWindow:
            cv2.imshow('Camera', frame)
    
        # Press 'q' to exit the loop
        if cv2.waitKey(1) == ord('q'):
            break
    

## Capture Livestream Video from webcam

### Use 'q' to close the window

In [3]:
# Open the default camera
cam = cv2.VideoCapture(0)

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))


# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('../dataset/stream_result/output_webcam.mp4', fourcc, 20.0, (frame_width, frame_height))


# START PREDICTION AND PRESENTATION
predict_video_stream(cam, out, verbose=False, showWindow=True, flipImage=True)


# Release the capture and writer objects
cam.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)



2024-11-25 11:34:10.170 python3[52561:3324672] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-11-25 11:34:12.738 python3[52561:3324672] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-25 11:34:12.738 python3[52561:3324672] +[IMKInputSession subclass]: chose IMKInputSession_Modern


-1

# Analyzing Video File 

In [11]:
# Open the default camera
cam = cv2.VideoCapture("../datasets/custom_ppe_videos/scene1.mp4")

# Get the default frame width and height
frame_width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('../datasets/stream_result/output_custom_video.mp4', fourcc, 20.0, (frame_width, frame_height))


# START PREDICTION AND PRESENTATION
predict_video_stream(cam, out, verbose=False, showWindow=True)


# Release the capture and writer objects
cam.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

2024-11-25 11:27:27.777 python3[52297:3316469] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-25 11:27:27.777 python3[52297:3316469] +[IMKInputSession subclass]: chose IMKInputSession_Modern


-1